## Install nilearn as it does not exist in the environment

In [ ]:
!pip install nilearn

## Import necessary packages

In [ ]:
import os
import glob
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from textwrap import wrap

from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.io import DataSink
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.base import \
    BaseInterface, TraitedSpec, File, \
    traits, BaseInterfaceInputSpec, SimpleInterface, InputMultiPath

## Dice Dissimilarity calculation nipype interface
#### - Inputs take each volume from 2 directories
#### - Output gives Dice score of 2 volumes

In [ ]:
class DiceDissimilarityInputSpec(BaseInterfaceInputSpec):
    volume1 = File(
        exists=True, mandatory=True, desc="Has to have the same dimensions as volume2."
    )
    volume2 = File(
        exists=True, mandatory=True, desc="Has to have the same dimensions as volume1."
    )


class DiceDissimilarityOutputSpec(TraitedSpec):
    dice = traits.Float(desc="dice dissimilarity")


class DiceDissimilarity(BaseInterface):
    input_spec = DiceDissimilarityInputSpec
    output_spec = DiceDissimilarityOutputSpec

    def _run_interface(self, runtime):
        nii1 = nb.load(self.inputs.volume1)
        data1 = np.asanyarray(nii1.dataobj).astype(int)
        data1[np.isnan(data1)] = 0
        
        nii2 = nb.load(self.inputs.volume2)
        data2 = np.asanyarray(nii2.dataobj).astype(int)
        data2[np.isnan(data2)] = 0
        
        dice_dissimilarity = 1-2*np.sum(data1==data2)/ (len(data1.flat) + len(data2.flat))

        self._dice = dice_dissimilarity
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs["dice"] = self._dice
        return outputs

## Plot Histogram nipype interface
#### - Inputs take all Dice scores previously calculated in the pipeline
#### - Outputs give .txt file of all Dice scores and histogram plot in png

In [ ]:
def plotHist(nii_files, maxvals=None, minvals=None):
    dice_array = []
    
    for in_nii_file in nii_files:
        dice_array.append(in_nii_file)

    arr_file = "array.txt"
    with open(arr_file, 'w+') as f:
        for line in dice_array:
            f.write(str(line) + "\n")
    mean = np.mean(dice_array)
    std = np.std(dice_array)
    minval = np.min(dice_array)
    maxval = np.max(dice_array)

    F = plt.figure(figsize=(8.3,9), facecolor=(1,1,1))
    F.tight_layout()
            
    ax1 = F.add_axes([0.12, 0.12, 0.8, 0.8], xlim=(np.min(dice_array), np.max(dice_array)), ylim=(0,170))           
    bin_edges = np.histogram_bin_edges(dice_array, bins='fd', range=(np.min(dice_array), np.max(dice_array)))
    ax1.hist(dice_array, bins=bin_edges)
    ax1.set_title('\n'.join(wrap(f'mean = {mean:.6f}, standard deviation = {std:.6f}, range = {minval:.6f} - {maxval:.6f}, subjects = {len(dice_array)}')))
    ax1.set_xlabel('DICE coefficient')
    ax1.set_ylabel('Number of occurences')
    ax1.xaxis.set_major_formatter(FormatStrFormatter('%.5f'))
    
    file_png = "plot.png"
    F.savefig(file_png,format='png')
    plt.clf()
    plt.close()
    del F
    return file_png, arr_file

class HistogramPlotInputSpec(TraitedSpec):
    in_files = InputMultiPath(
        exists=True, mandatory=True
    )

class HistogramPlotOutputSpec(TraitedSpec):
    plot = File(exists=True)
    arr = File(exists=True)

class HistogramPlot(BaseInterface):
    input_spec = HistogramPlotInputSpec
    output_spec = HistogramPlotOutputSpec
    
    def _run_interface(self, runtime):
        self._plot, self._arr = plotHist(self.inputs.in_files)
        
        runtime.returncode=0
        return runtime
    
    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs["plot"] = os.path.abspath(self._plot)
        outputs["arr"] = os.path.abspath(self._arr)
        return outputs

## Nipype workflow:
#### - Iterate through each outputs from FSL BET, FAST, FLIRT, FIRST
#### - Get all files from 2 directories and check if they have the same list of subjects
#### - Calculate the Dice Dissimilarity from curated files
#### - Generate file of Dice scores (.txt) and their distribution plot (.png) 

In [ ]:
def init_pattern_workflow(source_dir, target_dir, work_dir):
    patterns = ['brain.nii.gz', 'brain_seg.nii.gz', 'to_std_sub.nii.gz', 'all_fast_firstseg.nii.gz']
    wf = Workflow("workflow_pattern", base_dir=work_dir)
    wf.add_nodes([
        node for node in
        [init_subject_workflow(source_dir, target_dir, work_dir, pattern) for pattern in patterns]
        if node
    ])
    return wf


def init_subject_workflow(source_dir, target_dir, work_dir,pattern):
    wf = Workflow(pattern.replace('.nii.gz', ''), base_dir=os.path.join(work_dir, "workflow_pattern"))
    source_images = sorted(glob.glob(os.path.join(source_dir,  '*' + pattern)))
    source_subjects = sorted(list(set([os.path.split(path)[1] for path in source_images])))

    target_images = sorted(glob.glob(os.path.join(target_dir, '*' + pattern)))
    target_subjects = sorted(list(set([os.path.split(path)[1] for path in target_images])))

    # Double check if both directories have the same subjects
    source_path = sorted(
        os.path.join(os.path.dirname(image), subject)
        for image, subject in zip(source_images, source_subjects)
        if subject in target_subjects
    )
    target_path  = sorted(
        os.path.join(os.path.dirname(image), subject)
        for image, subject in zip(target_images, target_subjects)
        if subject in source_subjects
    )


    if not (target_path or source_path):
        print(f"No subjects found in: {os.path.join(target_dir, '*', pattern)} or {os.path.join(source_dir,'*', pattern)}")
        exit()

    source_target = Node(interface=IdentityInterface(fields=['source', 'target']),
                                                    name = 'source_target')                                               
    source_target.inputs.source = source_path
    source_target.inputs.target = target_path
    source_target.inputs.sort_filelist = True

    n_datasink = Node(
        interface=DataSink(
            base_directory=os.path.join(work_dir, pattern.replace('.nii.gz', ''))
        ),
        name='nipype_datasink'
    )

    BETDice = MapNode(interface=DiceDissimilarity(), 
                        name="dice_dissimilarity", 
                        iterfield=['volume1', 'volume2']
                    )

    plot_diff = Node(interface=HistogramPlot(), 
                        name="plot", 
                        iterfield=['in_files'])

    try:
        wf.connect([ 
            (source_target, BETDice, [('source', 'volume1'), ('target', 'volume2')]),
            (BETDice, plot_diff, [('dice', 'in_files')]),
            (plot_diff, n_datasink, [('plot', 'graph')]),
            (plot_diff, n_datasink, [('arr', 'array')])
            ])

    except(Exception) as err:
        print("Exception:", err)

    wf.run(
        plugin='MultiProc',
        plugin_args={
            'n_procs': 4
        }
    )

    return wf

## Specify the path to directories of images to compare (source_dir, target_dir)
#### Replace **"PATH_TO_IMAGE_DIR1" "PATH_TO_IMAGE_DIR2"** with your suitable directories.

## Specify new directory for outputs (work_dir)
#### Replace **"PATH_TO_OUTPUT_DIR"** with your suitable directories.

In [ ]:
# ensure directories are complete and absolute
source_dir = os.path.abspath("PATH_TO_IMAGE_DIR1")
target_dir = os.path.abspath("PATH_TO_IMAGE_DIR2")
work_dir = os.path.abspath("PATH_TO_OUTPUT_DIR")

wf = init_pattern_workflow(source_dir, target_dir, work_dir)